In [11]:
import matplotlib.pyplot as plt
import argparse
import io
import os
import torch
import numpy as np
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from torch.autograd import Variable
import cv2
import glob
import time
import torchvision.transforms as T
from torch import nn
from torch.nn import functional as F
import tensorrt as trt
import torchreid
from torch2trt import torch2trt
from torchreid.utils import FeatureExtractor
import torch.onnx

In [33]:
pixel_mean=[0.485, 0.456, 0.406]
pixel_std=[0.229, 0.224, 0.225]
paths = []
seuil = 12.0
min_dist = 2.46
max_dist = 15
fiablility = 100-(((distance-min_dist)/(max_dist-min_dist))*100)
preprocess = T.Compose([T.Resize((256,128)),
                        T.ToTensor(),
                        T.Normalize(mean=pixel_mean, std=pixel_std)])

def preprocess_and_infer(path_to_img):
        
        image = Image.open(path_to_img).convert('RGB')
        paths.append(image)
        image = preprocess(image)
        images = image.unsqueeze(0).to(device)
        with torch.no_grad():
            features = model(images)

        return features
    
    
if __name__ == "__main__": 
    #parser = argparse.ArgumentParser()
    #parser.parse_args()
    
    model = torch.load('extractor_osnet_x1.pth') 
    model.eval()
    device = torch.device('cuda')
    model.to(device)
    
    features = preprocess_and_infer("test.jpg")
    features2 = preprocess_and_infer("test2.jpg")
    distance = float(torch.dist(features,features2))
    print(distance)
    
    figure = plt.figure()
    figure.add_subplot(1,2, 1)
    plt.imshow(paths[0])
    figure.add_subplot(1,2, 2)
    plt.imshow(paths[1])
    
    if distance > seuil:
        figure.suptitle('no reID')
        plt.savefig('./results/output_.png')
    else:
        figure.suptitle('reID precision : '+str(round(100-(((distance-min_dist)/(max_dist-min_dist))*100),2))+' %')
        plt.savefig('./results/output_.png')
    
    plt.close(figure)
        
    



7.015953063964844
